In [1]:
import requests
import csv
import json
import re
import ast

In [13]:
en2twList = [["Housewares","家具"], ["Miscellaneous","小物件"], ["Wall-mounted","壁掛物"], ["Birthday","活動-玩家生日"], ["Bug-Off","活動-抓蟲大會"], ["Bunny Day","活動-復活節"], ["C.J.","俞司廷"], ["Crafting","DIY"], ["Dodo Airlines","Dodo公司"], ["Fishing Tourney","活動-釣魚大賽"], ["Flick","龍克斯"], ["Gulliver","呂游"], ["HHA","快樂家協會"], ["International Museum Day","活動-國際博物館日"], ["Mom","媽媽寄送"], ["Nintendo、Nook Shopping","購物-特別購物"], ["Nook Miles Shop","購物-哩數兌換"], ["Nook Shopping Promotion","購物-手游連動"], ["Nook Shopping Seasonal","購物-季節"], ["Nook's Cranny","Nook商店"], ["Rover","活動-勞動節"], ["Snowman","活動-雪人"], ["Starting items","起始物品"], ["Wedding Season","活動-六月新娘"], ["Air Conditioning","空調"], ["Animal","動物"], ["Arch","拱門"], ["Audio","音響"], ["Bathroom Things","浴室用品"], ["Bathtub","浴缸"], ["Beauty","美妝"], ["Bed","床鋪"], ["Chair","椅子"], ["Chest","櫃子"], ["Clock","時鐘"], ["Compass","羅盤"], ["Desk","檯子/桌子"], ["Dining","餐飲"], ["Dresser","梳妝台"], ["Easter","復活節"], ["Facility Decor","設施裝飾"], ["Fan","風扇"], ["Fireplace","壁爐"], ["Fish","魚"], ["Folk Craft Decor","民俗藝品"], ["Game Console","遊戲機"], ["Garden","花園"], ["Heating","加熱器"], ["Home Appliances","家用電器"], ["Hospital","醫院"], ["House Door Decor","屋門裝飾"], ["Insect","昆蟲"], ["Japanese Style","日式"], ["Kitchen","廚房"], ["Kitchen Things","廚房用品"], ["Lamp","燈"], ["Museum","博物館"], ["Musical Instrument","樂器"], ["Office","辦公室"], ["Outdoors Decor","戶外裝飾"], ["Plants","植物"], ["Playground","操場"], ["Ranch","牧場"], ["School","學校"], ["Screen","螢幕"], ["Sculpture","雕塑"], ["Seaside","海濱"], ["Seasonal Decor","季節性裝飾"], ["Sewing Table","縫紉桌"], ["Shelf","架子"], ["Shop","商店"], ["Sofa","沙發"], ["Space","太空"], ["Special Fish","特殊魚"], ["Special Insect","特殊昆蟲"], ["Sports","運動"], ["Study","讀書"], ["Supplies","耗材"], ["Table","桌子"], ["Toilet","廁所"], ["Toy","玩具"], ["TV","電視"], ["Vehicle","車輛"], ["Work Bench","工作台"], ["bathroom","浴室"], ["child's room","兒童房"], ["concert","音樂會"], ["den","書桌"], ["expensive","昂貴"], ["facility","設施"], ["fancy","豪華時髦的"], ["fitness","健身"], ["folk art","民間藝術"], ["freezing cold","雪季"], ["garage","車庫"], ["garden","花園"], ["horror","恐怖"], ["kitchen","廚房"], ["living room","客廳"], ["music","音樂"], ["ocean","海洋"], ["office","辦公室"], ["outdoors","戶外活動"], ["party","派對"], ["school","學校"], ["shop","商店"], ["space","太空"], ["zen-style","禪宗風格"]]
en2tw = {}
for en, tw in en2twList:
    en2tw[en] = tw

def en2twFn(en):
    if en in en2tw:
        return en2tw[en]
    return en

In [2]:
url = 'https://nookplaza.net/main.dart.js?v=9'
r = requests.get(url)
mainJs = r.text

In [3]:
# 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)
print(len(m))

m = [item.encode("utf-8").decode('unicode-escape') for item in m]
m = [ast.literal_eval(item) for item in m if not 'null' in item]
print(len(m))

nameMap = {}
for item in m:
    nameMap[item["en-us"].lower()] = item

5715
5713


In [4]:
nameMap['street organ']

{'de-de': 'Drehorgel',
 'en-gb': 'street organ',
 'en-us': 'street organ',
 'es-es': 'organillo en carro',
 'fr-ca': 'orgue de barbarie',
 'fr-fr': 'orgue de barbarie',
 'it-it': 'organo a rullo',
 'ja-jp': 'てまわしオルガン',
 'ko-kr': '배럴 오르간',
 'nl-nl': 'draaiorgel',
 'ru-ru': 'шарманка',
 'zh-cn': '手摇风琴',
 'zh-tw': '手搖風琴'}

In [22]:
url = 'https://api.nookplaza.net/items?category=Furniture'
r = requests.get(url)
data = r.json()['results']

In [27]:
idMap = {}
bodySet = set()
patternSet = set()
dictName = ['id', 'name_e', 'name_j', 'name_c', 'buy', 'sell', 'category', 'tag', 'themes', 'interact', 'size', 'obtainedFrom', 'DIY']
dictName += ['filename', 'bodyTitle', 'patternTitle', 'bodyCustomize', 'patternCustomize', 'bodys', 'pattrens', 'variations']

output = [['id', '名稱(英文)', '名稱(日文)', '名稱(中文)', '買價', '賣價', '分類', '標籤', '主題', '可互動', '大小', '取得方式', 'DIY', '圖片', '款式說明', '樣式說明', '款式改造', '樣式改造', '款式', '樣式', '檔名對照索引']]
for item in data:
    if item["category"] == "Art":
        continue
    name = nameMap[item['name'].lower()]
    nameMap[item['name'].lower()]['id'] = item['content']['internalID']
    idMap[item['content']['internalID']] = item

#     print(item['content'])
    item['content']['category_'] = en2twFn(item['content']['category'])
    item['content']['tag_'] = en2twFn(item['content']['tag'])
    item['content']['themes_'] = [en2twFn(theme) for theme in item['content']['themes']]
    item['content']['obtainedFrom_'] = [en2twFn(item) for item in item['content']['obtainedFrom']]
    
    # 名稱 買價 賣價
    row = [int(item['content']['internalID']), name['en-us'], name['ja-jp'], name['zh-tw'], item['content']['buy'], item['content']['sell']]

    # 分類 標籤 主題
    # 'category' 'tag' 'themes'
    if "None" in item['content']['themes_']:
        item['content']['themes_'].remove("None")
    row += [item['content']['category_'], item['content']['tag_'], '、'.join(item['content']['themes_'])]
    
    # 互動 大小
    row += [item['content']['interact'], '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])]

    # 取得方式 DIY
    if not 'dIY' in item['content']:
        print(item)
    row += ['、'.join(item['content']['obtainedFrom_']), item['content']['dIY']]

    # 圖片檔名
    row += [item['content']['filename']]
    
    # 改造-款式 樣式
    # bodyCustomize bodyColor bodyTitle; patternCustomize patternColor patternTitle
    row += [item['content']['bodyTitle'], item['content']['patternTitle'], item['content']['bodyCustomize'], item['content']['patternCustomize']]

    variations = {
        "bodys": {},
        "pattrens": {}
    }
    for variation in item['variations']:
        content = variation['content']
        filename = content['filename']
        bodyIndex, pattrenIndex = filename.split('_')[-2:]
        
        bodyColor = '' if content['bodyColor'] is None else content['bodyColor']
        patternColor = '' if content['patternColor'] is None else content['patternColor']
        
        variations['bodys'][bodyColor] = bodyIndex
        variations['pattrens'][patternColor] = pattrenIndex
        
        bodySet.add(bodyColor)
        patternSet.add(patternColor)

    row += ['、'.join(list(variations['bodys'].keys())), '、'.join(list(variations['pattrens'].keys())), variations]
       
#     print(row)
    output.append(row)
output[1:] = sorted(output[1:], key=lambda k: k[0])

In [28]:
with open('items.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    csvout.writerows(output)

jsonOutput = []
for row in output[0:]:
    j = {}
    for index, item in enumerate(row):
        j[dictName[index]] = item
    jsonOutput.append(j)

with open('furniture.json', 'wt') as fout:
    fout.write(json.dumps(jsonOutput))